In [1]:
import numpy as np
import rov
import matplotlib as mpl
import matplotlib.pyplot as plt

# What is the final project for AMath 352?

* Read a file containing an ROV configuration
* Read a series of Xbox controller requests
* Translate those controller requests into thrust settings for the ROV

# How are ROVs specified?

* The $x$ axis is presumed to lie along the length of the vehicle, with the positive $x$ axis pointing forward
* The $y$ axis is presumed to lie along the width of the vehicle, with the positive $y$ axis pointing to the port (left) side
* The $z$ axis runs along the height of the vehicle, with the positive $z$ axis pointing up
* The center of mass (engineers usually abbreviate this "CM") is presumed to be at the origin, i.e. $\text{CM} = (0, 0, 0)^T$
* The $i$-th thruster is located at the point $p_i=(x_i, y_i, z_i)$
* The $i$-th thruster is aimed in a direction $d_i=(f_i, s_i, h_i)$
    * The components of this vector are referred to as surge, sway, and heave
    * The magnitude of the thrust vector specifies the maximum thrust producible by that thruster

# How do ROVs (or airplanes, or cars, or . . .) move?

* Along each of the three coordinate directions (i.e. surge, sway, and heave)
* Additionally, a vehicle can spin around any of the three axes
    * Rotation about the $x$ axis is called roll
    * Rotation about the $y$ axis is called pitch
    * Rotation about the $z$ axis is called yaw
* The six degrees of freedom (surge, sway, heave, roll, pitch, and yaw) completely characterize the motion of the ROV in its own, real-time coordinate system
* Control engineers call this model a "6 DOF" model

# For this project, everything will be non-dimensionalized

* Each thruster can be assigned a thrust setting $t_i\in[-1, 1]$
* Each thruster produces motion equal to $t_i(f_i,s_i,h_i,r_i,p_i,y_i)^T$
* The motion of the ROV is given by

$$\begin{pmatrix}f_1 & f_2 & \ldots & f_n\cr
s_1 & s_2 & \ldots & s_n\cr
\vdots & \vdots & \ddots & \vdots\cr
y_1 & y_2 & \ldots & y_n\cr\end{pmatrix}
\begin{pmatrix}t_1\cr t_2\cr \vdots \cr t_n\cr\end{pmatrix}$$

* The main idea is to establish a right hand side vector $b$ and then solve (as best you can) 

$$At=b$$

* The columns of $A$ are given by

$$c_i=\begin{pmatrix}d_i\cr p_i\times d_i\cr\end{pmatrix}$$

* The vector $b$ is determined via the following algorithm
    * Start with the six component vector of current Xbox controller settings
    * Each component is a number between $-1$ and $1$, with $-1$ and $1$ both specifying "give me all you've got"
    * To get $b$ multiply each of the controller settings by the $1$-norm of the corresponding row of $A$.

# Solve $At=b$

* Many things can and will go wrong
    * This linear system can be underdetermined
    * It can be overdetermined
    * It can be simultaneously underdetermined and overdetermined
    * Scaling is required to get a good engineering solution
    * The solution may involve thrust settings that live outside $[-1, 1]$
* To deal with the out of range thrust settings
    * Find the setting most out of range
    * Set that thrust to $\pm 1$, depending on what its computed value is
    * Since it's no longer a variable
        * Subtract the projected thrust setting times the corresponding column of the matrix from the right hand side
        * Delete that column from the matrix
    * Attempt to resolve the linear problem
    * Continue until all of the thrust settings are within range or there's no linear system left

# How should one deal with scaling?

* Imagine "solving" the following equations

$$\begin{pmatrix}1\cr 1\cr\end{pmatrix}x=\begin{pmatrix}1\cr 0.1\cr\end{pmatrix}$$

* The least squares solution to this problem is

$$x=0.55$$

* This is fine, up to a point
* What if I am concerned about the relative error for each equation?
* Something needs to change
* Try scaling the 2nd equation by $\sqrt{10}$
* Then

$$\begin{pmatrix}1\cr \sqrt{10}\cr\end{pmatrix}x=\begin{pmatrix}1\cr 0.1\sqrt{10}\cr\end{pmatrix}$$

* The least squares solution to this modified problem is

$$x=\frac{2}{11}\approx0.18182$$

* Now the relative error in each equation is the exactly same, namely $\frac{9}{11}$
* This is exactly what needs to be done for the project, i.e. ensure that the solution treats all X-box controller requests the same

# What steps are required to complete the project?

* Write Python code to solve the problem
* Test it on the posted example files
* Schedule a 15 minute assessment with me
* Demonstrate your code on a problem you haven't seen before
* Enjoy the summer!

# Things your code MUST do to receive full credit

* Work on any combination of thrusters and 6 DOF inputs
* Scale the equations appropriately so that all Xbox controller inputs are treated equally in terms of relative error
* Specify thrusters in terms of position, toe angle, cant angle, and thrust rating
    * Toe angle is the angle by which the thruster is turned toward the centerline
    * Cant angle is the angle the thruster is rotated upward
    * Thrust rating is the fraction of thrust produced by the most powerful thruster
* Allow a weight vector to determine which specifications are most important

# Several test cases are posted on the syllabus

* Each file has the following format:
    * number of thrusters
    * x y z toe cant rating
    * fweight, sweight, hweight, rweight, pweight, yweight
    * number of moves
    * fforce, sforce, hforce, rtorque, ptorque, ytorque

* Compare your answers with those given below
* Let me know right away if you find an answer better than mine
* Ask lots of questions of me, the TAs, and your fellow students!

In [2]:
# Test 1 - A single thruster at the origin

test1 = rov.ROV([rov.Thruster()])
print("Thrust matrix = ", test1.thrustMatrix)
test1.move([1, 0, 0, 0, 0, 0])
test1.move([0, 1, 0, 0, 0, 0])
test1.move([0, 0, 1, 0, 0, 0])
test1.move([0, 0, 0, 1, 0, 0])
test1.move([0, 0, 0, 0, 1, 0])
test1.move([0, 0, 0, 0, 0, 1])

Thrust matrix =  [[ 1.]
 [-0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-0.]]

Desired Move =  [1, 0, 0, 0, 0, 0]
Computed thruster settings =  [1.]
Desired right hand side =  [1. 0. 0. 0. 0. 0.]
Right hand side achieved =  [1. 0. 0. 0. 0. 0.]

Desired Move =  [0, 1, 0, 0, 0, 0]
Computed thruster settings =  [0.]
Desired right hand side =  [0. 0. 0. 0. 0. 0.]
Right hand side achieved =  [0. 0. 0. 0. 0. 0.]

Desired Move =  [0, 0, 1, 0, 0, 0]
Computed thruster settings =  [0.]
Desired right hand side =  [0. 0. 0. 0. 0. 0.]
Right hand side achieved =  [0. 0. 0. 0. 0. 0.]

Desired Move =  [0, 0, 0, 1, 0, 0]
Computed thruster settings =  [0.]
Desired right hand side =  [0. 0. 0. 0. 0. 0.]
Right hand side achieved =  [0. 0. 0. 0. 0. 0.]

Desired Move =  [0, 0, 0, 0, 1, 0]
Computed thruster settings =  [0.]
Desired right hand side =  [0. 0. 0. 0. 0. 0.]
Right hand side achieved =  [0. 0. 0. 0. 0. 0.]

Desired Move =  [0, 0, 0, 0, 0, 1]
Computed thruster settings =  [0.]
Desired right hand side =  [0. 0. 0. 0.

In [3]:
# Test 2 - A single thruster out on the port side

test2 = rov.ROV([rov.Thruster([0, 1, 0])])
print("Thrust matrix = ", test2.thrustMatrix)
test2.move([1, 0, 0, 0, 0, 0])
test2.move([0, 1, 0, 0, 0, 0])
test2.move([0, 0, 1, 0, 0, 0])
test2.move([0, 0, 0, 1, 0, 0])
test2.move([0, 0, 0, 0, 1, 0])
test2.move([0, 0, 0, 0, 0, 1])
test2.move([1, 0, 0, 0, 0, 1])

Thrust matrix =  [[ 1.]
 [-0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-1.]]

Desired Move =  [1, 0, 0, 0, 0, 0]
Computed thruster settings =  [0.5]
Desired right hand side =  [1. 0. 0. 0. 0. 0.]
Right hand side achieved =  [ 0.5  0.   0.   0.   0.  -0.5]

Desired Move =  [0, 1, 0, 0, 0, 0]
Computed thruster settings =  [0.]
Desired right hand side =  [0. 0. 0. 0. 0. 0.]
Right hand side achieved =  [0. 0. 0. 0. 0. 0.]

Desired Move =  [0, 0, 1, 0, 0, 0]
Computed thruster settings =  [0.]
Desired right hand side =  [0. 0. 0. 0. 0. 0.]
Right hand side achieved =  [0. 0. 0. 0. 0. 0.]

Desired Move =  [0, 0, 0, 1, 0, 0]
Computed thruster settings =  [0.]
Desired right hand side =  [0. 0. 0. 0. 0. 0.]
Right hand side achieved =  [0. 0. 0. 0. 0. 0.]

Desired Move =  [0, 0, 0, 0, 1, 0]
Computed thruster settings =  [0.]
Desired right hand side =  [0. 0. 0. 0. 0. 0.]
Right hand side achieved =  [0. 0. 0. 0. 0. 0.]

Desired Move =  [0, 0, 0, 0, 0, 1]
Computed thruster settings =  [-0.5]
Desired right hand side 

In [4]:
# Test 3 - Balanced thrusters on both sides

portThruster = rov.Thruster([0.0, 1.0, 0.0])
starboardThruster = rov.Thruster([0.0, -1.0, 0.0])
test3 = rov.ROV([portThruster, starboardThruster])
print("Thrust matrix = ", test3.thrustMatrix)
test3.move([1, 0, 0, 0, 0, 1])
test3.move([1, 1, 0, 0, 0, 0.75])
test3.move([1, 0, 1, 0, 0, 0.5])
test3.move([1, 0, 0, 1, 0, 0.25])
test3.move([1, 0, 0, 0, 1, 0.1])
test3.move([0, 0, 0, 0, 0, -0.8])
test3.move([0.5, 0, 0, 0, 0, 1])

Thrust matrix =  [[ 1.  1.]
 [-0.  0.]
 [ 0.  0.]
 [ 0. -0.]
 [ 0.  0.]
 [-1.  1.]]

Desired Move =  [1, 0, 0, 0, 0, 1]
Computed thruster settings =  [0. 1.]
Desired right hand side =  [2. 0. 0. 0. 0. 2.]
Right hand side achieved =  [1. 0. 0. 0. 0. 1.]

Desired Move =  [1, 1, 0, 0, 0, 0.75]
Computed thruster settings =  [0.14285714 1.        ]
Desired right hand side =  [2.  0.  0.  0.  0.  1.5]
Right hand side achieved =  [1.14285714 0.         0.         0.         0.         0.85714286]

Desired Move =  [1, 0, 1, 0, 0, 0.5]
Computed thruster settings =  [0.33333333 1.        ]
Desired right hand side =  [2. 0. 0. 0. 0. 1.]
Right hand side achieved =  [1.33333333 0.         0.         0.         0.         0.66666667]

Desired Move =  [1, 0, 0, 1, 0, 0.25]
Computed thruster settings =  [0.6 1. ]
Desired right hand side =  [2.  0.  0.  0.  0.  0.5]
Right hand side achieved =  [1.6 0.  0.  0.  0.  0.4]

Desired Move =  [1, 0, 0, 0, 1, 0.1]
Computed thruster settings =  [0.81818182 1.  

In [5]:
# Test 4 - An X-wing fighter

test4 = rov.ROV([rov.Thruster([0, 1, 0.5]),
                 rov.Thruster([0, 1, -0.5]),
                 rov.Thruster([0, -1, 0.5]),
                 rov.Thruster([0, -1, -0.5])], [1, 1, 1, 1, 1, 1])
print("Thrust matrix = ", test4.thrustMatrix)
test4.move([1, 0, 0, 0, 0, 1])
test4.move([1, 0, 0, 0, 1, 0])
test4.move([0, 0, 0, 0, 1, 1])
test4.move([1, 0, 0, 0, 1, 1])
test4.move([-0.5, 0, 0, 0, 0.8, 1])
test4.move([0.4, 0, 0, 0, 0, 0])
test4.move([0.4, 0, 0, 0, 0.4, 0.4])

Thrust matrix =  [[ 1.   1.   1.   1. ]
 [-0.  -0.   0.   0. ]
 [ 0.   0.   0.   0. ]
 [ 0.   0.  -0.   0. ]
 [ 0.5 -0.5  0.5 -0.5]
 [-1.  -1.   1.   1. ]]

Desired Move =  [1, 0, 0, 0, 0, 1]
Computed thruster settings =  [ 4.53246652e-17 -4.53246652e-17  1.00000000e+00  1.00000000e+00]
Desired right hand side =  [4. 0. 0. 0. 0. 4.]
Right hand side achieved =  [2. 0. 0. 0. 0. 2.]

Desired Move =  [1, 0, 0, 0, 1, 0]
Computed thruster settings =  [1.         0.33333333 1.         0.33333333]
Desired right hand side =  [4. 0. 0. 0. 2. 0.]
Right hand side achieved =  [ 2.66666667e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  6.66666667e-01 -5.55111512e-17]

Desired Move =  [0, 0, 0, 0, 1, 1]
Computed thruster settings =  [-0.33333333 -1.          1.          0.33333333]
Desired right hand side =  [0. 0. 0. 0. 2. 4.]
Right hand side achieved =  [2.22044605e-16 0.00000000e+00 0.00000000e+00 0.00000000e+00
 6.66666667e-01 2.66666667e+00]

Desired Move =  [1, 0, 0, 0, 1, 1]
Computed th

In [6]:
# Test 5 - Two balanced thrusters with toe and cant angles
 
portThruster = rov.Thruster([0.0, 1.0, 0.0], 2.0, 5.0)
starboardThruster = rov.Thruster([0.0, -1.0, 0.0], 2.0, 5.0)
test5 = rov.ROV([portThruster, starboardThruster])
print("Thrust matrix = ", test5.thrustMatrix)
test5.move([1, 0, 0, 0, 0, 1])
test5.move([1, 1, 0, 0, 0, 0.75])
test5.move([1, 0, 1, 0, 0, 0.5])
test5.move([1, 0, 0, 1, 0, 0.25])
test5.move([1, 0, 0, 0, 1, 0.1])
test5.move([0, 0, 0, 0, 0, -0.8])
test5.move([0, 0, 0, 1, 0, 0])
test5.move([0.5, 0, 0, 0, 0, 1])
test5.move([0.5, 0, 0, 0, 0, 0])
test5.move([0.25, 0, 0, 0, 0.25, 0.25])

Thrust matrix =  [[ 0.99558784  0.99558784]
 [-0.03476669  0.03476669]
 [ 0.08715574  0.08715574]
 [ 0.08715574 -0.08715574]
 [ 0.          0.        ]
 [-0.99558784  0.99558784]]

Desired Move =  [1, 0, 0, 0, 0, 1]
Computed thruster settings =  [0.0011944 1.       ]
Desired right hand side =  [1.99117569 0.         0.         0.         0.         1.99117569]
Right hand side achieved =  [ 0.99677698  0.03472517  0.08725984 -0.08705164  0.          0.99439871]

Desired Move =  [1, 1, 0, 0, 0, 0.75]
Computed thruster settings =  [0.12440216 1.        ]
Desired right hand side =  [1.99117569 0.06953339 0.         0.         0.         1.49338176]
Right hand side achieved =  [ 1.11944112  0.03044164  0.09799811 -0.07631338  0.          0.87173457]

Desired Move =  [1, 0, 1, 0, 0, 0.5]
Computed thruster settings =  [0.35592527 1.        ]
Desired right hand side =  [1.99117569 0.         0.17431149 0.         0.         0.99558784]
Right hand side achieved =  [ 1.34994271  0.02239235  0.11

In [7]:
# Test 6 - Four balanced thrusters, i.e. the "707" configuration

test6 = rov.ROV([rov.Thruster([0, 2, 0]),
                 rov.Thruster([0, 1, 0]),
                 rov.Thruster([0, -1, 0]),
                 rov.Thruster([0, -2, 0])])
print("Thrust matrix = ", test6.thrustMatrix)
test6.move([1, 0, 0, 0, 0, 0])
test6.move([1, 0, 0, 0, 0, 1])
test6.move([0.25, 0, 0, 0, 0, 0])
test6.move([0.57, 0, 0, 0, 0, 0.57])

Thrust matrix =  [[ 1.  1.  1.  1.]
 [-0. -0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0. -0. -0.]
 [ 0.  0.  0.  0.]
 [-2. -1.  1.  2.]]

Desired Move =  [1, 0, 0, 0, 0, 0]
Computed thruster settings =  [1. 1. 1. 1.]
Desired right hand side =  [4. 0. 0. 0. 0. 0.]
Right hand side achieved =  [4. 0. 0. 0. 0. 0.]

Desired Move =  [1, 0, 0, 0, 0, 1]
Computed thruster settings =  [-1.  1.  1.  1.]
Desired right hand side =  [4. 0. 0. 0. 0. 6.]
Right hand side achieved =  [2. 0. 0. 0. 0. 4.]

Desired Move =  [0.25, 0, 0, 0, 0, 0]
Computed thruster settings =  [0.25 0.25 0.25 0.25]
Desired right hand side =  [1. 0. 0. 0. 0. 0.]
Right hand side achieved =  [ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00 -1.11022302e-16]

Desired Move =  [0.57, 0, 0, 0, 0, 0.57]
Computed thruster settings =  [-0.7   0.98  1.    1.  ]
Desired right hand side =  [2.28 0.   0.   0.   0.   3.42]
Right hand side achieved =  [2.28 0.   0.   0.   0.   3.42]


In [8]:
# Test 7 - Oh no! The inboard thruster on the starboard side has just failed
 
test7 = rov.ROV([rov.Thruster([0, 2, 0]),
                 rov.Thruster([0, 1, 0]),
                 rov.Thruster([0, -2, 0])])
print("Thrust matrix = ", test7.thrustMatrix)
test7.move([1, 0, 0, 0, 0, 0])
test7.move([1, 0, 0, 0, 0, 1])
test7.move([0.25, 0, 0, 0, 0, 0])
test7.move([0.57, 0, 0, 0, 0, 0.57])

Thrust matrix =  [[ 1.  1.  1.]
 [-0. -0.  0.]
 [ 0.  0.  0.]
 [ 0.  0. -0.]
 [ 0.  0.  0.]
 [-2. -1.  2.]]

Desired Move =  [1, 0, 0, 0, 0, 0]
Computed thruster settings =  [0.53846154 1.         1.        ]
Desired right hand side =  [3. 0. 0. 0. 0. 0.]
Right hand side achieved =  [ 2.53846154  0.          0.          0.          0.         -0.07692308]

Desired Move =  [1, 0, 0, 0, 0, 1]
Computed thruster settings =  [-1.  1.  1.]
Desired right hand side =  [3. 0. 0. 0. 0. 5.]
Right hand side achieved =  [1. 0. 0. 0. 0. 3.]

Desired Move =  [0.25, 0, 0, 0, 0, 0]
Computed thruster settings =  [0.20192308 0.23076923 0.31730769]
Desired right hand side =  [0.75 0.   0.   0.   0.   0.  ]
Right hand side achieved =  [7.50000000e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 1.11022302e-16]

Desired Move =  [0.57, 0, 0, 0, 0, 0.57]
Computed thruster settings =  [-0.73823529  1.          1.        ]
Desired right hand side =  [1.71 0.   0.   0.   0.   2.85]
Right hand sid

In [9]:
# Test 8 - The starship Enterprise, sort of

test8 = rov.ROV([rov.Thruster([-1, 1, 1]),
                 rov.Thruster([-1, -1, 1]),
                 rov.Thruster([0, 0, 0])])
print("Thrust matrix = ", test8.thrustMatrix)
test8.move([1, 0, 0, 0, 0, 0])
test8.move([0, 0, 0, 0, 1, 0])
test8.move([0, 0, 0, 0, 0, 1])
test8.move([0.2, 0, 0, 0, 0.5, 0.5])

Thrust matrix =  [[ 1.  1.  1.]
 [-0.  0. -0.]
 [ 0.  0.  0.]
 [ 0. -0.  0.]
 [ 1.  1.  0.]
 [-1.  1. -0.]]

Desired Move =  [1, 0, 0, 0, 0, 0]
Computed thruster settings =  [0.25 0.25 1.  ]
Desired right hand side =  [3. 0. 0. 0. 0. 0.]
Right hand side achieved =  [1.5 0.  0.  0.  0.5 0. ]

Desired Move =  [0, 0, 0, 0, 1, 0]
Computed thruster settings =  [ 0.66666667  0.66666667 -1.        ]
Desired right hand side =  [0. 0. 0. 0. 2. 0.]
Right hand side achieved =  [0.33333333 0.         0.         0.         1.33333333 0.        ]

Desired Move =  [0, 0, 0, 0, 0, 1]
Computed thruster settings =  [-1.  1.  0.]
Desired right hand side =  [0. 0. 0. 0. 0. 2.]
Right hand side achieved =  [0. 0. 0. 0. 0. 2.]

Desired Move =  [0.2, 0, 0, 0, 0.5, 0.5]
Computed thruster settings =  [-5.55111512e-17  1.00000000e+00 -4.00000000e-01]
Desired right hand side =  [0.6 0.  0.  0.  1.  1. ]
Right hand side achieved =  [0.6 0.  0.  0.  1.  1. ]


In [10]:
# Test 9 - The starship Enterprise again, emphasize forward motion by a factor of 10

test9 = rov.ROV([rov.Thruster([-1, 1, 1]),
                 rov.Thruster([-1, -1, 1]),
                 rov.Thruster([0, 0, 0])],
                 [10, 1, 1, 1, 1, 1])
print("Thrust matrix = ", test9.thrustMatrix)
test9.move([1, 0, 0, 0, 0, 0])
test9.move([0, 0, 0, 0, 1, 0])
test9.move([0, 0, 0, 0, 0, 1])
test9.move([0.2, 0, 0, 0, 0.5, 0.5])

Thrust matrix =  [[ 1.  1.  1.]
 [-0.  0. -0.]
 [ 0.  0.  0.]
 [ 0. -0.  0.]
 [ 1.  1.  0.]
 [-1.  1. -0.]]

Desired Move =  [1, 0, 0, 0, 0, 0]
Computed thruster settings =  [0.76923077 0.76923077 1.        ]
Desired right hand side =  [3. 0. 0. 0. 0. 0.]
Right hand side achieved =  [2.53846154e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.53846154e+00 2.22044605e-16]

Desired Move =  [0, 0, 0, 0, 1, 0]
Computed thruster settings =  [ 0.52380952  0.52380952 -1.        ]
Desired right hand side =  [0. 0. 0. 0. 2. 0.]
Right hand side achieved =  [4.76190476e-02 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.04761905e+00 1.11022302e-16]

Desired Move =  [0, 0, 0, 0, 0, 1]
Computed thruster settings =  [-1.00000000e+00  1.00000000e+00  1.66533454e-16]
Desired right hand side =  [0. 0. 0. 0. 0. 2.]
Right hand side achieved =  [1.66533454e-16 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 2.00000000e+00]

Desired Move =  [0.2, 0, 0, 0, 0.5, 0.5]
Computed thruster setting